In [ ]:
import os
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import chromadb

current_dir = os.path.dirname(os.path.abspath("__file__"))
file_path = os.path.join(current_dir, "books", "智慧型手機使用手冊.txt")
persistent_directory = os.path.join(current_dir, "db", "chroma_db_v3")
try:
    if not os.path.exists(persistent_directory):
        if not os.path.exists(file_path):
            raise FileNotFoundError(
                f"檔案{file_path}不存在,請檢查路徑"
            )

        loader = TextLoader(file_path=file_path,encoding='utf-8')
        documents = loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=200)
        docs = text_splitter.split_documents(documents)
        #print(len(docs))
        #print(docs[0].page_content)
        embeddings = HuggingFaceEmbeddings(
            model_name="BAAI/bge-m3"
        )

        client = chromadb.PersistentClient(path=persistent_directory)
        db = Chroma.from_documents(
            docs,
            embedding=embeddings,
            client = client,
            collection_name="smartphone_manual"
        )




    else:
        print("向量資料庫已經存在,不需要初始化")
except Exception as error:
    print(error)

向量資料庫已經存在,不需要初始化


In [1]:
query = "請問手機硬體規格如何?"

embeddings = HuggingFaceEmbeddings(
            model_name="BAAI/bge-m3"
        )

client = chromadb.PersistentClient(path=persistent_directory)
db = Chroma.from_documents(
            docs,
            embedding=embeddings,
            client = client,
            collection_name="smartphone_manual"
        )

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

relevant_docs = retriever.invoke(query)
for doc in relevant_docs:
    print(doc.page_content)
    print("=" * 60)

NameError: name 'HuggingFaceEmbeddings' is not defined